In [21]:
import sqlite3
from math import pi

def save_output(fname, dbpath):
    i=0
    my_data = {}
    with (open(fname,'r')) as f:
        for line in f:
            if i > 0:
                tokens = line.split(',')
                crack_id = int(tokens[0])
                theta = float(tokens[1])
                x = float(tokens[2])
                y = float(tokens[3])
                z = float(tokens[4])
                dadN2D = float(tokens[5])
                dadN3Dline = float(tokens[6])
                dadNpath = float(tokens[7].rstrip())
                if crack_id in my_data:
                    my_data[crack_id].append((theta,x,y,z,dadN2D,dadN3Dline,dadNpath))
                else:
                    my_data[crack_id] = []
                    my_data[crack_id].append((theta,x,y,z,dadN2D,dadN3Dline,dadNpath))
            i+=1

    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):#(6,61,3):
        conn.execute('alter table data_%d'%r+' add column dadN2D float')
        conn.execute('alter table data_%d'%r+' add column dadN3Dline float')
        conn.execute('alter table data_%d'%r+' add column dadNpath float')

        cursor = conn.execute('select crack_id, arr_index, theta from data_%d order by crack_id asc, theta desc'%r)
        for row in cursor:
            crack_id = row[0]
            if crack_id < 9 and crack_id > 0:
                arr_index = row[1]
                theta_node_ = row[2]
                this_crack = my_data[crack_id]
                index = 0

                # Don't flip theta, I checked it in Paraview
                theta_node = theta_node_ 

                while index < len(this_crack) and this_crack[index][0] <= theta_node:
                    before_theta = this_crack[index][0]
                    before_dadN2D = this_crack[index][4]
                    before_dadN3Dline = this_crack[index][5]
                    before_dadNpath = this_crack[index][6]
                    index += 1

                if theta_node > 0 and index < len(this_crack):
                    after_theta = this_crack[index][0]
                    after_dadN2D = this_crack[index][4]
                    after_dadN3Dline = this_crack[index][5]
                    after_dadNpath = this_crack[index][6]
                else:
                    after_theta = before_theta
                    after_dadN2D = before_dadN2D
                    after_dadN3Dline = before_dadN3Dline
                    after_dadNpath = before_dadNpath

                final_theta = (before_theta + after_theta) / 2.0
                final_dadN2D = (before_dadN2D + after_dadN2D) / 2.0
                final_dadN3Dline = (before_dadN3Dline + after_dadN3Dline) / 2.0
                final_dadNpath = (before_dadNpath + after_dadNpath) / 2.0

                conn.execute('update data_%d set dadN2D=%f,dadN3Dline=%f,dadNpath=%f where crack_id=%d and arr_index=%d' \
                             %(r,final_dadN2D,final_dadN3Dline,final_dadNpath,crack_id,arr_index))
    conn.commit()
    conn.close()
    print 'done'

In [20]:
save_output('../data-smaller-r/crack_front_growth_rates_1500ppcf.csv', '../../big-data-smaller-r/base.db')

done


In [6]:
3.141592 - 0.477639

2.6639530000000002

In [43]:
conn.close()